In [ ]:
#13.6 GB


from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import multi30k, Multi30k
from typing import Iterable, List
import json




!pip install -U torchdata
!pip install -U spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 5.9 MB/s eta 0:00:00
  Attempting uninstall: typer
    Found existing installation: typer 0.9.0
    Uninstalling typer-0.9.0:
      Successfully uninstalled typer-0.9.0
  Attempting uninstall: spacy
    Found existing installation: spacy 3.6.1
    Uninstalling spacy-3.6.1:
      Successfully uninstalled spacy-3.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.6.0 requires spacy<3.7.0,>=3.6.0, but you have spacy 3.7.0 which is incompatible.
2023-10-03 08:51:38.515078: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To e

In [ ]:
!git clone "https://github.com/anoopkunchukuttan/indic_nlp_library"
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git
!pip install Morfessor
# The path to the local git repo for Indic NLP library
INDIC_NLP_LIB_HOME=r"/content/indic_nlp_library"

# The path to the local git repo for Indic NLP Resources
INDIC_NLP_RESOURCES="/content/indic_nlp_resources"
import sys
sys.path.append(r'{}'.format(INDIC_NLP_LIB_HOME))
from indicnlp import common
common.set_resources_path(INDIC_NLP_RESOURCES)
from indicnlp import loader
loader.load()
from indicnlp.tokenize import indic_tokenize


from indicnlp.normalize.indic_normalize import BaseNormalizer

Cloning into 'indic_nlp_library'...
remote: Enumerating objects: 1399, done.
remote: Counting objects: 100% (180/180), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 1399 (delta 135), reused 148 (delta 121), pack-reused 1219
Receiving objects: 100% (1399/1399), 9.57 MiB | 14.62 MiB/s, done.
Resolving deltas: 100% (745/745), done.
Cloning into 'indic_nlp_resources'...
remote: Enumerating objects: 139, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 139 (delta 2), reused 2 (delta 0), pack-reused 126
Receiving objects: 100% (139/139), 149.77 MiB | 19.67 MiB/s, done.
Resolving deltas: 100% (53/53), done.
Updating files: 100% (28/28), done.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pairs = ['English-Bengali',
        'English-Gujarati',
        'English-Hindi',
        'English-Kannada',
        'English-Malayalam',
        'English-Tamil',
        'English-Telgu']

pair =  'English-Hindi'

In [ ]:
with open('/content/drive/MyDrive/NLP/train_data1.json', 'r') as file: # Replace this path with the dataset path in your local machine
    data = json.load(file)

# Process JSON data
source_sentences_train = []
target_sentences_train = []

# source_sentences_val = []
# target_sentences_val = []

id_train = []
#id_val = []


normalizer = BaseNormalizer("hi", remove_nuktas=False)
# normalizer=factory.get_normalizer("hi",remove_nuktas)
#output_text=normalizer.normalize(target)

tokenizer_dummy = get_tokenizer('spacy', language='en_core_web_sm')
before_trans = 0

for language_pair, language_data in data.items():
    if(language_pair == pair):
      print(f"Language Pair: {language_pair}")
      for data_type, data_entries in language_data.items():
          print(f"  Data Type: {data_type}")
          for entry_id, entry_data in data_entries.items():
              source = entry_data["source"]
              target = entry_data["target"]
              before_trans+=1
              tokenized_source = tokenizer_dummy(source)
              length = len(tokenized_source)
              remove_if = ["information", "s.", "parenting", "aroma", "j.", "myspace", "pandora", "spotify", \
                          "sad", "lyrics", "."]

              if length ==0 or tokenized_source[0] in remove_if or length>=100:
                continue


              source_sentences_train.append(source.lower())
              target_sentences_train.append(target)
              id_train.append(entry_id)

            #   if (data_type == "Validation"):
            #     #source_sentences_val.append(source)
            #     #target_sentences_val.append(target)
            #     #id_val.append(entry_id)
            #   else:


Language Pair: English-Hindi
  Data Type: Train


In [ ]:
#train_zip = zip(source_sentences_train, target_sentences_train)
train_zip = []
for i in range(len(source_sentences_train)):
    train_zip.append([source_sentences_train[i], target_sentences_train[i]])

train_zip[0]

['cancel everything on my calendar', 'मेरे कैलेंडर पर सब कुछ रद्द करें']

In [ ]:

after_trans = len(source_sentences_train)
print("Before trans", before_trans)
print("After trans", after_trans)


Before trans 80797
After trans 80762


In [ ]:
EMB_SIZE = 200
import torchtext.vocab
glove_vectors = torchtext.vocab.GloVe(name = '6B', dim = EMB_SIZE)

.vector_cache/glove.6B.zip: 862MB [02:40, 5.38MB/s]                           
100%|█████████▉| 399999/400000 [00:38<00:00, 10466.52it/s]


In [ ]:
SRC_LANGUAGE = 'en'
TGT_LANGUAGE = 'hi'  #change for other langs? nah just treat as a constant name for others

# Place-holders
token_transform = {}
vocab_transform = {}
train_data_transform = {}

token_transform[SRC_LANGUAGE] = get_tokenizer('spacy', language='en_core_web_sm')
token_transform[TGT_LANGUAGE] = indic_tokenize.trivial_tokenize
train_data_transform[SRC_LANGUAGE] = source_sentences_train
train_data_transform[TGT_LANGUAGE] = target_sentences_train


In [ ]:
print(token_transform[TGT_LANGUAGE](target_sentences_train[0]))
print(token_transform[SRC_LANGUAGE](source_sentences_train[0]))

['मेरे', 'कैलेंडर', 'पर', 'सब', 'कुछ', 'रद्द', 'करें']
['cancel', 'everything', 'on', 'my', 'calendar']


In [ ]:
# helper function to yield list of tokens
def yield_tokens(data_iter: Iterable, language: str) -> List[str]:
    language_index = {SRC_LANGUAGE: 0, TGT_LANGUAGE: 1}

    for data_sample in data_iter:
        #yield token_transform[language](data_sample[language_index[language]])
        yield token_transform[language](data_sample)

# Define special symbols and indices
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
# Make sure the tokens are in order of their indices to properly insert them in vocab
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    # Training data Iterator
    #train_iter = train_zip
    train_iter = train_data_transform[ln]
    # Create torchtext's Vocab object
    vocab_transform[ln] = build_vocab_from_iterator(yield_tokens(train_iter, ln),
                                                    min_freq=1,
                                                    specials=special_symbols,
                                                    special_first=True,
                                                    max_tokens = 15000)

# Set ``UNK_IDX`` as the default index. This index is returned when the token is not found.
# If not set, it throws ``RuntimeError`` when the queried token is not found in the Vocabulary.
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
  vocab_transform[ln].set_default_index(UNK_IDX)

In [ ]:
print(vocab_transform[SRC_LANGUAGE].get_itos())
print(vocab_transform[TGT_LANGUAGE].get_itos())

['<unk>', '<pad>', '<bos>', '<eos>', 'the', '.', ',', 'of', 'and', 'in', 'to', 'is', 'a', 'for', 'this', 'are', 'that', 'it', '-', 'you', 'on', 'with', 'be', 'from', 'by', 'as', 'or', 'can', 'at', 'i', 'have', 'so', 'was', 'not', 'which', 'also', 'will', 'there', ')', '(', 'has', 'an', 'we', 'one', 'if', 'what', '`', 'do', 'all', 'but', 'should', "'s", 'then', 'your', 'its', 'these', 'time', 'after', 'india', 'some', 'more', 'other', 'they', 'here', 'he', 'his', 'like', 'their', 'been', '?', 'now', 'when', 'two', 'my', 'only', 'many', 'me', 'about', 'may', 'people', 'were', 'get', 'very', 'most', 'first', 'such', '"', ':', 'water', 'during', 'who', 'used', 'because', 'any', 'out', 'how', 'up', 'had', 'into', 'than', 'place', 'use', 'state', 'temple', 'where', 'just', 'take', 'new', 'them', 'through', 'going', 'city', 'see', 'right', 'world', 'disease', 'make', 'number', 'no', 'food', 'day', 'body', 'three', 'made', 'well', 'national', 'want', 'indian', 'let', 'while', 'even', 'list', '

In [ ]:
pretrained_embeddings = glove_vectors.get_vecs_by_tokens(vocab_transform[TGT_LANGUAGE].get_itos())


In [ ]:
print(len(vocab_transform[SRC_LANGUAGE]))
len(vocab_transform[TGT_LANGUAGE])

15000


15000

In [ ]:
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
import math
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# helper Module that adds positional encoding to the token embedding to introduce a notion of word order.
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Seq2Seq Network
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

In [ ]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [ ]:
torch.manual_seed(0)

SRC_VOCAB_SIZE = len(vocab_transform[SRC_LANGUAGE])
TGT_VOCAB_SIZE = len(vocab_transform[TGT_LANGUAGE])
NHEAD = 8  #8
FFN_HID_DIM = 256
BATCH_SIZE = 64 #32
NUM_ENCODER_LAYERS = 2  #3
NUM_DECODER_LAYERS = 2  #3
EMB_SIZE = 200

transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)

transformer.tgt_tok_emb.embedding.weight.data = pretrained_embeddings


for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(DEVICE)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

optimizer = torch.optim.Adam(transformer.parameters(), lr=0.001, betas=(0.9, 0.98), eps=1e-9)  #lr=lr=0.0001

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(transformer):,} trainable parameters')

The model has 10,396,024 trainable parameters


In [ ]:
from torch.nn.utils.rnn import pad_sequence

# helper function to club together sequential operations
def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

# function to add BOS/EOS and create tensor for input sequence indices
def tensor_transform(token_ids: List[int]):
    return torch.cat((torch.tensor([BOS_IDX]),
                      torch.tensor(token_ids),
                      torch.tensor([EOS_IDX])))

# src and tgt language text transforms to convert raw strings into tensors indices
text_transform = {}
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    text_transform[ln] = sequential_transforms(token_transform[ln], #Tokenization
                                               vocab_transform[ln], #Numericalization
                                               tensor_transform) # Add BOS/EOS and create tensor


# function to collate data samples into batch tensors
def collate_fn(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_batch.append(text_transform[SRC_LANGUAGE](src_sample.rstrip("\n")))
        tgt_batch.append(text_transform[TGT_LANGUAGE](tgt_sample.rstrip("\n")))

    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch, tgt_batch

In [ ]:
from torch.utils.data import DataLoader

def train_epoch(model, optimizer, clip):
    model.train()
    losses = 0

    train_dataloader = DataLoader(train_zip, batch_size=BATCH_SIZE,
                                  collate_fn=collate_fn)
    batches = 1;
    log_idx = 200;
    for src, tgt in train_dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        optimizer.zero_grad()

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()
        losses += loss.item()

        batches += 1

        if batches % log_idx == 0:
          print('| Batch {:5d} | Loss {:8.3f}|'.format(batches, losses/batches))

    return losses / len(list(train_dataloader))

In [ ]:
from torchtext.data.metrics import bleu_score

def evaluate(model):
    model.eval()
    losses = 0

    val_dataloader = DataLoader(valid_dp, batch_size=BATCH_SIZE,
                                 collate_fn=collate_fn)
    predictions, ground_truth = [], []

    counter = 1;
    log_idx = 200;

    with torch.no_grad():
      print("-"*30)
      print("VALIDATION")
      print("-"*30)
      for src, tgt in val_dataloader:
          src = src.to(DEVICE)
          tgt = tgt.to(DEVICE)

          tgt_input = tgt[:-1, :]

          src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

          logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

          tgt_out = tgt[1:, :]
          loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
          losses += loss.item()

          # BLEU Score
          # predictions.append(logits.cpu().detach())
          # ground_truth.append(tgt_out.cpu().detach())

          counter += 1

          if counter % log_idx == 0:
              print('| Batch {:5d} | Loss {:8.3f}|'.format(counter, losses/counter))

    # predictions = torch.cat(predictions).cpu().detach()
    # ground_truth = torch.cat(ground_truth).cpu().detach()
    # _bleu = bleu_score(predictions, ground_truth)

    return losses / len(list(val_dataloader))

In [ ]:
from timeit import default_timer as timer
NUM_EPOCHS = 20
Clip = 1
#best_valid_loss = float('inf')

for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss = train_epoch(transformer, optimizer, Clip)
    end_time = timer()
    #torch.save(transformer.state_dict(), 'best-mt-model.pt')
    #val_loss = evaluate(transformer)

    # if val_loss < best_valid_loss:
    #     best_valid_loss = val_loss
    #     torch.save(transformer.state_dict(), 'best-mt-model.pt')
    # print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))


torch.save(transformer.state_dict(), pair+'model.pt')


/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:4999: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


| Batch   200 | Loss    6.063|
| Batch   400 | Loss    5.592|
| Batch   600 | Loss    5.318|
| Batch   800 | Loss    5.111|
| Batch  1000 | Loss    4.952|
| Batch  1200 | Loss    4.821|
| Batch   200 | Loss    3.909|
| Batch   400 | Loss    3.847|
| Batch   600 | Loss    3.782|
| Batch   800 | Loss    3.715|
| Batch  1000 | Loss    3.662|
| Batch  1200 | Loss    3.611|
| Batch   200 | Loss    3.197|
| Batch   400 | Loss    3.163|
| Batch   600 | Loss    3.125|
| Batch   800 | Loss    3.084|
| Batch  1000 | Loss    3.057|
| Batch  1200 | Loss    3.031|
| Batch   200 | Loss    2.797|
| Batch   400 | Loss    2.780|
| Batch   600 | Loss    2.757|
| Batch   800 | Loss    2.730|
| Batch  1000 | Loss    2.716|
| Batch  1200 | Loss    2.701|
| Batch   200 | Loss    2.564|
| Batch   400 | Loss    2.550|
| Batch   600 | Loss    2.534|
| Batch   800 | Loss    2.513|
| Batch  1000 | Loss    2.505|
| Batch  1200 | Loss    2.495|
| Batch   200 | Loss    2.396|
| Batch   400 | Loss    2.389|
| Batch 

In [ ]:
import pickle

with open('/content/drive/MyDrive/NLP/Pickles/' + pair + '.pkl', 'wb') as file:
    # Serialize and save the object to the file
    pickle.dump(transformer, file)


In [ ]:
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys


# actual function to translate input sentence into target language
def translate(model: torch.nn.Module, src_sentence: str):
    model.eval()
    src = text_transform[SRC_LANGUAGE](src_sentence).view(-1, 1)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(
        model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
    return " ".join(vocab_transform[TGT_LANGUAGE].lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")

In [ ]:
with open('/content/drive/MyDrive/NLP/test_data1_final.json', 'r') as file: # Replace this path with the dataset path in your local machine
    data = json.load(file)

# Process JSON data

source_sentences_val = []
target_sentences_val = []

id_train = []
id_val = []



for language_pair, language_data in data.items():
    if(language_pair == pair):
      print(f"Language Pair: {language_pair}")
      for data_type, data_entries in language_data.items():
          print(f"  Data Type: {data_type}")
          for entry_id, entry_data in data_entries.items():
              source = entry_data["source"]
              #target = entry_data["target"]
              source_sentences_val.append(source.lower())
            #target_sentences_val.append(target)
              id_val.append(entry_id)
            #   else:
            #     source_sentences_train.append(source)
            #     #target_sentences_train.append(target)
            #     id_train.append(entry_id)

Language Pair: English-Hindi
  Data Type: Test


In [ ]:
# test_zip = []
# for i in range(len(source_sentences_val)):
#     train_zip.append([source_sentences_val[i], target_sentences_val[i]])

# test_zip[0]

In [ ]:
LetusC = source_sentences_val[5:10]
for i in LetusC:
  print(i)
  print(translate(transformer, i))

#print(translate(transformer, "Eine Gruppe von Menschen steht vor einem Iglu ."))


print(len(source_sentences_val))

as such this much is fixed that whenever i go on a holiday , then i harbor a desire of complete peace and rest .
 ऐसा इतना निर्धारित है कि जब भी मैं छुट्टी पर जाते हूं , तो मैं सुकून देता हूं और शेष की इच्छा रखता हूं । 
so because of such considerations the designer designers of c++ and many languages decided that if you want multiplication to happen you have to say multiplication so you have to explicitly indicate the multiplication operator.
 ऐसे <unk> के कारण C + और कई भाषाओं के <unk> के कारण <unk> ने तय किया कि यदि आप अंग को गुणा करना चाहते हैं तो आपको स्पष्ट रूप से कहना होगा कि गुणन ऑपरेटर ऑपरेटर ऑपरेटर को स्पष्ट रूप से
due to this, the agricultural productivity of maharashtra is generally low as compared to the national averages of various crops.
 इसके कारण महाराष्ट्र की कृषि उत्पादकता प्रायः विभिन्न फसलों की अपेक्षा कम होती है । 
specialists from around the world are unanimous in classifying brazil as among the countries richest in fish, and therefore an ideal destination for sp

In [ ]:
import csv
filename = '/content/drive/MyDrive/NLP/CSVs/'+"MT_" + pair +  ".csv"
# field names
fields = ['ID', 'Translation']

# data rows of csv file
rows = []


for i in range(len(source_sentences_val)):
  rows.append([id_val[i], translate(transformer, source_sentences_val[i])])

# name of csv file

# writing to csv file
with open(filename, 'w') as csvfile:
    # creating a csv writer object
    csvwriter = csv.writer(csvfile)

    # writing the fields
    csvwriter.writerow(fields)

    # writing the data rows
    csvwriter.writerows(rows)

